In [1]:
from collections import Counter
import re
import random

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost
from xgboost import XGBClassifier
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, BatchNormalization
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import RMSprop
from treeinterpreter import treeinterpreter as ti
import shap
import eli5
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel

%matplotlib inline

Using TensorFlow backend.
/home/lucia/.virtualenvs/odsc1/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is

In [2]:
def read_data(filename):
    """
    Function to read data from text to dataframe.
    :param filename: path and name of the file to read
    :type filename: str
    :return: dataframe of the file
    :rtype: pandas.core.frame.DataFrame
    """
    with open(filename, encoding='utf-8-sig') as txt:
        data = txt.read()
    data = [x.split('\t') for x in data.split('\n')]
    data = pd.DataFrame(data[1::], columns=data[0])
    return data

In [3]:
filename_demographics = '10000_patients/PatientCorePopulatedTable.txt'
demographics = read_data(filename_demographics)

filename_admissions = '10000_patients/AdmissionsCorePopulatedTable.txt'
admissions = read_data(filename_admissions)

filename_diagnoses = '10000_patients/AdmissionsDiagnosesCorePopulatedTable.txt'
diagnoses = read_data(filename_diagnoses)

In [4]:
# join all data
data = diagnoses.merge(admissions, how='left', on=['PatientID', 'AdmissionID']).merge(
    demographics, how='left', on='PatientID')

null_data = [i for i in data.index if not data.PatientID[i]]
data.loc[null_data]

# delete null data
data = data.drop(null_data, axis=0).reset_index(drop=True)

In [5]:
def convert_dt(date):
    if not date:
        return None
    return datetime.strptime(date, '%Y-%m-%d %H:%M:%S.%f')

def get_deltadays(date1, date2):
    if not date1 or not date2:
        return None
    return (convert_dt(date1) - convert_dt(date2)).days

In [6]:
# calculate age at admission
data['admission_age'] = [get_deltadays(data.AdmissionStartDate[i], data.PatientDateOfBirth[i])/365 for i in 
                        data.index]

# calculate the number of admitted days
data['admitted_days'] = [get_deltadays(data.AdmissionEndDate[i], data.AdmissionStartDate[i]) for i in 
                        data.index]

# engineer ICD10 main diagnoses
data['icd10'] = [x[0:2] for x in data.PrimaryDiagnosisCode]

# encode gender
data['gender'] = [0 if x == 'Male' else 1 for x in data.PatientGender]

# long admissions
data['long_admission'] = [0 if x <= 2 else 1 for x in data.admitted_days]

data = data.sort_values(by=['PatientID', 'admission_age']).reset_index(drop=True)

In [7]:
data.head()

,PatientID,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,AdmissionStartDate,AdmissionEndDate,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,PatientLanguage,PatientPopulationPercentageBelowPoverty,admission_age,admitted_days,icd10,tumor,gender,long_admission
0,0002C021-6DDD-433A-B9A0-2D26DF0C6AC0,1,F51.04,Psychophysiologic insomnia,1981-05-21 05:21:14.380,1981-05-26 11:13:06.313,Male,1954-11-18 14:15:38.637,White,Married,English,14.63,26.520548,5,F5,0,0,1
1,0002C021-6DDD-433A-B9A0-2D26DF0C6AC0,2,M05.33,Rheumatoid heart disease with rheumatoid arthr...,1983-03-22 05:04:47.540,1983-03-26 04:24:25.987,Male,1954-11-18 14:15:38.637,White,Married,English,14.63,28.356164,3,M0,0,0,1
2,0002C021-6DDD-433A-B9A0-2D26DF0C6AC0,3,B95.3,Streptococcus pneumoniae as the cause of disea...,1997-03-26 20:04:15.043,1997-03-30 13:08:15.633,Male,1954-11-18 14:15:38.637,White,Married,English,14.63,42.380822,3,B9,0,0,1
3,0002C021-6DDD-433A-B9A0-2D26DF0C6AC0,4,F68.12,Factitious disorder with predominantly physica...,2003-09-09 04:17:31.027,2003-09-27 08:13:34.593,Male,1954-11-18 14:15:38.637,White,Married,English,14.63,48.838356,18,F6,0,0,1
4,0002C021-6DDD-433A-B9A0-2D26DF0C6AC0,5,F64,Gender identity disorders,2004-03-27 01:01:29.530,2004-04-07 14:52:36.153,Male,1954-11-18 14:15:38.637,White,Married,English,14.63,49.386301,11,F6,0,0,1


In [8]:
interesting_columns = ['gender', 'PatientRace', 'PatientMaritalStatus', 'PatientLanguage', 
                       'PatientPopulationPercentageBelowPoverty', 'admission_age', 'admitted_days', 'icd10']
object_columns = [x for x in data.select_dtypes(['O']).columns if x in interesting_columns]
for col in object_columns:
    data[col] = data[col].astype('category')

cat_cols = data.select_dtypes(['category']).columns
data['cat_' + cat_cols] = data[cat_cols].apply(lambda x: x.cat.codes)
data.head()

,PatientID,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,AdmissionStartDate,AdmissionEndDate,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,...,admitted_days,icd10,tumor,gender,long_admission,cat_PatientRace,cat_PatientMaritalStatus,cat_PatientLanguage,cat_PatientPopulationPercentageBelowPoverty,cat_icd10
0,0002C021-6DDD-433A-B9A0-2D26DF0C6AC0,1,F51.04,Psychophysiologic insomnia,1981-05-21 05:21:14.380,1981-05-26 11:13:06.313,Male,1954-11-18 14:15:38.637,White,Married,...,5,F5,0,0,1,3,1,0,588,45
1,0002C021-6DDD-433A-B9A0-2D26DF0C6AC0,2,M05.33,Rheumatoid heart disease with rheumatoid arthr...,1983-03-22 05:04:47.540,1983-03-26 04:24:25.987,Male,1954-11-18 14:15:38.637,White,Married,...,3,M0,0,0,1,3,1,0,588,103
2,0002C021-6DDD-433A-B9A0-2D26DF0C6AC0,3,B95.3,Streptococcus pneumoniae as the cause of disea...,1997-03-26 20:04:15.043,1997-03-30 13:08:15.633,Male,1954-11-18 14:15:38.637,White,Married,...,3,B9,0,0,1,3,1,0,588,15
3,0002C021-6DDD-433A-B9A0-2D26DF0C6AC0,4,F68.12,Factitious disorder with predominantly physica...,2003-09-09 04:17:31.027,2003-09-27 08:13:34.593,Male,1954-11-18 14:15:38.637,White,Married,...,18,F6,0,0,1,3,1,0,588,46
4,0002C021-6DDD-433A-B9A0-2D26DF0C6AC0,5,F64,Gender identity disorders,2004-03-27 01:01:29.530,2004-04-07 14:52:36.153,Male,1954-11-18 14:15:38.637,White,Married,...,11,F6,0,0,1,3,1,0,588,46


These are the features we want to consider as columns:
- 'admission_age',
- 'admitted_days', 
- 'icd10', 
- 'gender', 
- 'cat_PatientRace',
- 'cat_PatientMaritalStatus', 
- 'cat_PatientLanguage',
- 'PatientPopulationPercentageBelowPoverty'

In [9]:
subjects, last_age, tot_admissions, long_admissions = [], [], [], []
gender, race, status, language, poverty, past_diagnosis = [], [], [], [], [], []
no_admissions = []

for subj in set(data.PatientID):
    temp = data.loc[data.PatientID == subj]
    
    temp_index = temp.index.tolist()
    
    if len(temp_index) > 1:

        s_last_age = temp.admission_age[temp_index[-2]]
        s_tot_admissions = len(temp_index) - 1
        s_long_admissions = sum(temp.long_admission.tolist()[0:-1]) / s_tot_admissions
        s_gender = temp.gender[temp_index[-1]]
        s_race = temp.cat_PatientRace[temp_index[-1]]
        s_status = temp.cat_PatientMaritalStatus[temp_index[-1]]
        s_language = temp.cat_PatientLanguage[temp_index[-1]]
        s_poverty = float(temp.PatientPopulationPercentageBelowPoverty[temp_index[-1]])
        s_past_diagnosis = ' '.join(temp.icd10.tolist()[0:-1])

        s_no_admissions = 1 if temp.admission_age[temp_index[-1]] - temp.admission_age[temp_index[-2]] < 7 else 0

        subjects.append(subj)

        # features
        last_age.append(s_last_age)
        tot_admissions.append(s_tot_admissions)
        long_admissions.append(s_long_admissions)
        gender.append(s_gender)
        race.append(s_race)
        status.append(s_status)
        language.append(s_language)
        poverty.append(s_poverty)
        past_diagnosis.append(s_past_diagnosis)

        # label

        no_admissions.append(s_no_admissions)

# Predict: is the patient going to be healthy for the next x years?

In [10]:
df_healthy = pd.DataFrame({'last_age': last_age, 'tot_admissions': tot_admissions, 
                         'long_admissions': long_admissions, 'gender': gender, 'race': race, 'status': status,
                         'language': language, 'poverty': poverty, 'past_diagnosis': past_diagnosis})

In [11]:
# split train and test
X_train, X_test, y_train, y_test = train_test_split(df_healthy, no_admissions, test_size=0.33, 
                                                    random_state=42, stratify=no_admissions)

In [12]:
count_vect = CountVectorizer(lowercase=False)
count_vect.fit(X_train.past_diagnosis)

col_names = [f'ICD10_{x}' for x in count_vect.vocabulary_]

past_diagnosis_train = count_vect.transform(X_train.past_diagnosis)
past_diagnosis_train = pd.DataFrame(past_diagnosis_train.todense(), columns = col_names, index=X_train.index)

past_diagnosis_valid = count_vect.transform(X_test.past_diagnosis)
past_diagnosis_valid = pd.DataFrame(past_diagnosis_valid.todense(), columns = col_names, index=X_test.index)

X_train1 = pd.concat([X_train, past_diagnosis_train], axis=1).drop(['past_diagnosis'], axis=1)
X_test1 = pd.concat([X_test, past_diagnosis_valid], axis=1).drop(['past_diagnosis'], axis=1)

In [13]:
# balance the classes
class_weight = compute_class_weight("balanced", np.unique(y_train), y_train)
class_weight_dict = dict(enumerate(class_weight))
print(class_weight_dict)

{0: 0.9768262737875998, 1: 1.024299967814612}


In [14]:
def fit_and_predict(model, X, y, Xtest, ytest):
    model = model.fit(X, y)
    test_preds = model.predict(Xtest)
    test_scores = model.predict_proba(Xtest)
    
    print('Confusion matrix for the model')
    print(confusion_matrix(y_test, test_preds))
    prec, rec, thresholds = precision_recall_curve(ytest, test_scores[:, 1])
    pr_auc = auc(rec, prec)
    print(f"Precision-Recall AUC: {pr_auc}")
    plt.plot(rec, prec)
    plt.title(f"Precision-Recall AUC: {pr_auc}")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.show()
    
    return model

### White box model

In [ ]:
model_lr = LogisticRegression(class_weight=class_weight_dict, max_iter=10000)
model_lr = fit_and_predict(model_lr, X_train1, y_train, X_test1, y_test)

In [ ]:
# getting interpretation of a white box model
lr_coef = model_lr.coef_[0]
sorted_idx = lr_coef.argsort()
# get only values that show significance
sorted_idx = [i for i in sorted_idx if lr_coef[i] > 0.5 or lr_coef[i] < -0.5]

sorted_features = [X_train1.columns[i] for i in sorted_idx]
sorted_importance = [model_lr.coef_[0][i] for i in sorted_idx]

y_ticks = np.arange(0, len(sorted_features))
fig, ax = plt.subplots(figsize=(7, 10))
ax.barh(y_ticks, sorted_importance)
ax.set_yticklabels(sorted_features)
ax.set_yticks(y_ticks)
ax.set_title("Feature Importances")
fig.tight_layout()
plt.show()

### Simple black box model

In [ ]:
model_rf = RandomForestClassifier(class_weight=class_weight_dict)
model_rf = fit_and_predict(model_rf, X_train1, y_train, X_test1, y_test)

In [ ]:
# tree interpreter
instances = X_test1.loc[[x for x in X_test1.index if x in [4350, 9408]]]
print(f"Instances prediction: {model_rf.predict(instances)}")

In [ ]:
prediction, bias, contributions = ti.predict(model_rf, instances)

In [ ]:
for i in range(len(instances)):
    print("Instance", i)
    print("Bias (trainset mean)", bias[i])
    print("Feature contributions:")
    for c, feature in sorted(zip(contributions[i][:, 1], 
                                 X_test1.columns), 
                             key=lambda x: -abs(x[0])):
        if abs(c) > 0.01:
            print(feature, round(c, 2))
    print("-"*20)

In [ ]:
# Feature importance
sorted_idx = model_rf.feature_importances_.argsort()
# get only values that show significance
sorted_idx = [i for i in sorted_idx if model_rf.feature_importances_[i] > 0.005]

sorted_features = [X_train1.columns[i] for i in sorted_idx]
sorted_importance = [model_rf.feature_importances_[i] for i in sorted_idx]

y_ticks = np.arange(0, len(sorted_features))
fig, ax = plt.subplots(figsize=(12, 9))
ax.barh(y_ticks, sorted_importance)
ax.set_yticklabels(sorted_features)
ax.set_yticks(y_ticks)
ax.set_title("Feature Importances")
fig.tight_layout()
plt.show()

In [ ]:
import eli5

In [ ]:
eli5.show_weights(model_rf)

In [ ]:
print('Actual Label:', y_test[0])
print('Predicted Label:', model_rf.predict_proba(X_test1.loc[X_test1.index[0:1]])[0][1])
eli5.show_prediction(model_rf, X_test1.loc[X_test1.index[0:1]], 
                     feature_names=list(X_test1.columns),
                     show_feature_values=True)

#### Skater

In [ ]:
interpreter = Interpretation(training_data=X_test1, training_labels=y_test, 
                             feature_names=list(X_test1.columns))
im_model = InMemoryModel(model_rf.predict_proba, examples=X_train1, 
                         # target_names=['$50K or less', 'More than $50K']
                        )

In [ ]:
plots = interpreter.feature_importance.plot_feature_importance(im_model, ascending=True, n_samples=10)

In [ ]:
# partial dependency plot

r = interpreter.partial_dependence.plot_partial_dependence(['tot_admissions'], im_model, grid_resolution=50, 
                                                           grid_range=(0,1),  
                                                           with_variance=True, figsize = (6, 4))
yl = r[0][1].set_ylim(0, 1) 

In [ ]:
r = interpreter.partial_dependence.plot_partial_dependence(['last_age'], im_model, grid_resolution=50, 
                                                           grid_range=(0,1),  
                                                           with_variance=True, figsize = (6, 4))
yl = r[0][1].set_ylim(0, 1) 

## Fitting a more complex model

In [ ]:
class_weight_dict_xgb = (len(y_train) - sum(y_train)) / sum(y_train)

model_xgb = XGBClassifier(objective='binary:logistic', scale_pos_weight=class_weight_dict_xgb)
model_xgb = fit_and_predict(model_xgb, X_train1, y_train, X_test1, y_test)

### In-house feature importance

In [ ]:
fig = plt.figure(figsize = (20, 30))
title = fig.suptitle("Default Feature Importances from XGBoost", fontsize=14)

ax1 = fig.add_subplot(2,2, 1)
xgboost.plot_importance(model_xgb, importance_type='weight', ax=ax1)
t=ax1.set_title("Feature Importance - Feature Weight")

ax2 = fig.add_subplot(2,2, 2)
xgboost.plot_importance(model_xgb, importance_type='gain', ax=ax2)
t=ax2.set_title("Feature Importance - Split Mean Gain")

ax3 = fig.add_subplot(2,2, 3)
xgboost.plot_importance(model_xgb, importance_type='cover', ax=ax3)
t=ax3.set_title("Feature Importance - Sample Coverage")

### Skater feature importance

In [ ]:
interpreter = Interpretation(training_data=X_test1, training_labels=y_test, 
                             feature_names=list(X_test1.columns))
im_model = InMemoryModel(model_xgb.predict_proba, examples=X_train1)

In [ ]:
plots = interpreter.feature_importance.plot_feature_importance(im_model, ascending=True) # 400 seconds

In [ ]:
r = interpreter.partial_dependence.plot_partial_dependence(['poverty'], im_model, grid_resolution=50, 
                                                           grid_range=(0,1), with_variance=True, figsize = (6, 4))
yl = r[0][1].set_ylim(0, 1) # 42 seconds

In [ ]:
plots_list = interpreter.partial_dependence.plot_partial_dependence([('tot_admissions', 'status')], 
                                                                    im_model, grid_range=(0,1), 
                                                                    figsize=(12, 5),
                                                                    grid_resolution=100) # 110 seconds

In [ ]:
# find good predicted
predictions = model_xgb.predict(X_test1)
pred0_index = [i for i,x in enumerate(predictions) if x == y_test[i] and y_test[i] == 0]
pred1_index = [i for i,x in enumerate(predictions) if x == y_test[i] and y_test[i] == 1]

### LIME with Skater

In [ ]:
'''
Skater can leverage LIME to explain model predictions. 
Typically, its LimeTabularExplainer class helps in explaining predictions on tabular (i.e. matrix) data. 
For numerical features, it perturbs them by sampling from a Normal(0,1) 
and doing the inverse operation of mean-centering and scaling, 
according to the means and stds in the training data. For categorical features, 
it perturbs by sampling according to the training distribution, 
and making a binary feature that is 1 when the value is the same as the instance being explained. 
The explain_instance() function generates explanations for a prediction. 
First, we generate neighborhood data by randomly perturbing features from the instance. 
We then learn locally weighted linear (surrogate) 
models on this neighborhood data to explain each of the classes in an interpretable way.
'''

In [ ]:
from skater.core.local_interpretation.lime.lime_tabular import LimeTabularExplainer

In [ ]:
exp = LimeTabularExplainer(X_test1.values, feature_names=list(X_test1.columns), 
                           discretize_continuous=True)

In [ ]:
model_xgb_value = XGBClassifier(objective='binary:logistic', scale_pos_weight=class_weight_dict_xgb)
model_xgb_value.fit(X_train1.values, y_train)

In [ ]:
print('Actual Label:', y_test[pred0_index[0]])
print('Predicted Label:', predictions[pred0_index[0]])
exp.explain_instance(X_test1.loc[pred0_index[0]].values, 
                     model_xgb_value.predict_proba).show_in_notebook()

In [ ]:
print('Actual Label:', y_test[pred1_index[0]])
print('Predicted Label:', predictions[pred1_index[0]])
exp.explain_instance(X_test1.loc[pred1_index[0]].values, 
                     model_xgb_value.predict_proba).show_in_notebook()

In [ ]:
# TREPAN in skater
# The implementation also generates a fidelity score to quantify tree based surrogate model’s 
# approximation to the Oracle. Ideally, the score should be 0 for truthful explanation 
# both globally and locally

In [ ]:
surrogate_explainer = interpreter.tree_surrogate(oracle=im_model, seed=42)

In [ ]:
surrogate_explainer.fit(X_train1, y_train, use_oracle=True, prune='pre', scorer_type='f1')

In [ ]:
pd.DataFrame([('X'+str(idx), feature) 
                   for (idx, feature) in enumerate(X_train1.columns)]).T

In [ ]:
from skater.util.dataops import show_in_notebook
from graphviz import Source
from IPython.display import SVG

In [ ]:
graph = Source(surrogate_explainer.plot_global_decisions(colors=['coral', 'darkturquoise'], 
                                          file_name='test_tree_pre.png').to_string())
svg_data = graph.pipe(format='svg')
with open('dtree_structure.svg','wb') as f:
    f.write(svg_data)
SVG(svg_data)

### Use SHAP

In [ ]:
explainer = shap.TreeExplainer(model_xgb)
shap_values = explainer.shap_values(X_test1)

In [ ]:
# shap for base prediction
# The following is the expected probability of something being 
# classified as Class 1. explainer.expected_value[0] gives the 
# expected probability of something being classfied as Class 0. Any # SHAP value contributes towards or against this base expected 
# probability, which is calcultated for the dataset, not for the 
# model.
print('Expected Value:', explainer.expected_value)
pd.DataFrame(shap_values).head()

In [ ]:
shap.initjs()
shap.summary_plot(shap_values, X_test1, plot_type="bar")

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[pred0_index[0],:], X_test1.iloc[pred0_index[0],:])

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[pred1_index[0],:], X_test1.iloc[pred1_index[0],:])

In [ ]:
shap.force_plot(explainer.expected_value, 
                shap_values[:1000,:], X_test1.loc[:1000,:])

In [ ]:
shap.summary_plot(shap_values, X_test1, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, X_test1)

In [ ]:
shap.dependence_plot(ind='tot_admissions', interaction_index='tot_admissions',
                     shap_values=shap_values, 
                     features=X_test1,  
                     display_features=X_test1)

In [ ]:
shap.dependence_plot(ind='last_age', interaction_index='last_age',
                     shap_values=shap_values, 
                     features=X_test1,  
                     display_features=X_test1)

In [ ]:
shap.dependence_plot(ind='tot_admissions', interaction_index='last_age', 
                     shap_values=shap_values, features=X_test1, 
                     display_features=X_test1)

## Neural Network 

In [15]:
def build_dense_model(initial_length):
    """
    Function to create a multilayer deep fully connected neural network
    :param initial_length: length of the input vectors
    :type initial_length: int
    :return: model according to input size
    :rtype: tensorflow.python.keras.engine.training.Model
    """
    inputs = Input(shape=(initial_length,), name="input")
    # normalized the batches
    x = BatchNormalization(name='input_bn')(inputs)
    x = Dense(256, activation="softmax", name="dense1")(x)
    x = Dropout(0.5, name="dropout1")(x)
    x = Dense(128, activation="softmax", name="dense2")(x)
    x = Dropout(0.5, name="dropout2")(x)
    x = Dense(256, activation="softmax", name="dense3")(inputs)
    x = Dropout(0.2, name="dropout3")(x)
    output = Dense(2, activation="softmax", name="output")(x)

    # Compile model and create model summary
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss="binary_crossentropy", optimizer=RMSprop(learning_rate=0.0001), metrics=["accuracy"])
    # model.summary()
    return model


In [16]:
dense = build_dense_model(X_train1.shape[1])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [17]:
early_stop = EarlyStopping(monitor="val_loss", patience=5, verbose=False)
checkpoint = ModelCheckpoint('model_dense.h5', verbose=0, monitor='val_loss',save_best_only=True, mode='auto')  

history = dense.fit(
    x=X_train1, 
    y=to_categorical(y_train),
    validation_split=0.2,
    batch_size=64,
    epochs=3, #1000,
    verbose=1,
    class_weight=class_weight_dict,
    callbacks=[early_stop, checkpoint],
    shuffle=True,
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 5092 samples, validate on 1273 samples
Epoch 1/3
5092/5092 [==============================] - 1s 171us/step - loss: 0.6910 - accuracy: 0.4853 - val_loss: 0.6890 - val_accuracy: 0.5027
Epoch 2/3
5092/5092 [==============================] - 0s 92us/step - loss: 0.6886 - accuracy: 0.4943 - val_loss: 0.6862 - val_accuracy: 0.5153
Epoch 3/3
5092/5092 [==============================] - 1s 107us/step - loss: 0.6859 - accuracy: 0.5487 - val_loss: 0.6833 - val_accuracy: 0.5962


In [18]:
dense1 = build_dense_model(X_train1.shape[1])
dense1.load_weights('model_dense.h5')

In [19]:
test_scores = dense1.predict(X_test1)
test_preds = [0 if x < 0.5 else 1 for x in test_scores[:, 1]]

print(confusion_matrix(y_test, test_preds))
precisions, recalls, thresholds = precision_recall_curve(y_test, test_scores[:, 1])
pr_auc = auc(recalls, precisions)
print(pr_auc)

[[ 433 1171]
 [ 122 1409]]
0.6463664930798853


**GradientExplainer**
Explains a model using expected gradients (an extension of integrated gradients).

Expected gradients an extension of the integrated gradients method (Sundararajan et al. 2017), a feature attribution method designed for differentiable models based on an extension of Shapley values to infinite player games (Aumann-Shapley values). Integrated gradients values are a bit different from SHAP values, and require a single reference value to integrate from. As an adaptation to make them approximate SHAP values, expected gradients reformulates the integral as an expectation and combines that expectation with sampling reference values from the background dataset. This leads to a single combined expectation of gradients that converges to attributions that sum to the difference between the expected model output and the current output.

**DeepExplainer**
Meant to approximate SHAP values for deep learning models.

This is an enhanced version of the DeepLIFT algorithm (Deep SHAP) where, similar to Kernel SHAP, we approximate the conditional expectations of SHAP values using a selection of background samples. Lundberg and Lee, NIPS 2017 showed that the per node attribution rules in DeepLIFT (Shrikumar, Greenside, and Kundaje, arXiv 2017) can be chosen to approximate Shapley values. By integrating over many backgound samples DeepExplainer estimates approximate SHAP values such that they sum up to the difference between the expected model output on the passed background samples and the current model output (f(x) - E[f(x)]).

from https://shap.readthedocs.io/en/latest/

Note: For tf.keras users, SHAP still does not support tf 2.0.
see https://github.com/slundberg/shap/issues/885

Use shap.GradientExplainer

In [23]:
explainer = shap.DeepExplainer(dense1, X_train1)
shap_values = explainer.shap_values(X_test1)

KeyError: 0

In [ ]:
# find good predicted
pred0_index = [i for i,x in enumerate(test_preds) if x == y_test[i] and y_test[i] == 0]
pred1_index = [i for i,x in enumerate(test_preds) if x == y_test[i] and y_test[i] == 1]

In [ ]:
exp.explain_instance(X_test1.loc[pred0_index[0]].values, 
                     model_xgb_value.predict_proba).show_in_notebook()

In [ ]:
# explain predictions of the model on four images
e = shap.DeepExplainer(model, background)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(x_test[1:5])

# plot the feature attributions
shap.image_plot(shap_values, -x_test[1:5])